<img src="https://files.cdn.thinkific.com/file_uploads/133292/images/d57/1b0/1a9/P03.jpg">

`MCSDSC02P03V3______`

`MCSDSELIMC02P03V2250621`

`MCSDSELIMC02P03V1130621`

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Замечаний снова нет, проект принят!

## Постановка задачи

Наша задача обработать информацию по данным случаев страхование:
есть ли зависимость между выплатами и:
* возрастной группой
* случаем страхования
* количеством дохода 

С помощью данных выводов мы сможем понять, дискриминирует страховая компания какие-то случаи и каких-то людей или нет

## Импорт данных и библиотек

Импортируем библиотеки, прочитаем файл `insurance.csv` и получим общую информацию о данных.

In [1]:
import pandas as pd
import numpy as np
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem

bo_data = pd.read_csv('../../data/insurance.csv')

Столбцы содержат следующую информацию:

* id - уникальный индетификатор пользователя
* perc_of_compensation_paid - процент выплаченной компенсации
* age_in_days - возраст в днях
* age_id - цифровой индетификатор возраста
* age_group - возрастная группа
* income - доход
* count_3-6_months_late - счетчик за последние 3-6 месяцев
* count_6-12_months_late - счетчик за последние 6-12 месяцев
* count_more_than_12_months_late - счетчик за последний год и более
* cause - причина


### Выводы

У нас есть таблица по страховым случаям клиентов размером в 2301 строку с данными о доходах, выплаченной компенсации, возрастной группе и тд

## Предобработка данных

**Обработка пропусков**

Проверим данные на наличие пропусков, просуммировав их.

In [2]:
bo_data['income'].isna().sum()

65

Заполним пропуски в столбце `income`.

In [3]:
bo_data['income'].fillna(bo_data['income'].median(), inplace=True)

Проверяем, что пропусков в `income` не осталось.

In [4]:
bo_data['income'].isna().sum()

0

**Замена типа данных**

Проверяем, что в столбцах `age_in_days` и `income` вещественный тип данных.

In [5]:
bo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2301 entries, 0 to 2300
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              2301 non-null   int64  
 1   perc_of_compensation_paid       2301 non-null   float64
 2   age_in_days                     2301 non-null   float64
 3   age_id                          2301 non-null   int64  
 4   age_group                       2301 non-null   object 
 5   income                          2301 non-null   float64
 6   count_3-6_months_late           2301 non-null   int64  
 7   count_6-12_months_late          2301 non-null   int64  
 8   count_more_than_12_months_late  2301 non-null   int64  
 9   cause                           2301 non-null   object 
dtypes: float64(3), int64(5), object(2)
memory usage: 179.9+ KB


Для этих столбцов нужен целочисленный тип, поэтому переведем их, а также избавимся от отрицательных значений в `income`.

In [6]:
bo_data['age_in_days'].astype('int64')

0       27384
1       23735
2       17170
3       16068
4       10591
        ...  
2296    17525
2297    21906
2298    25925
2299    23370
2300    21914
Name: age_in_days, Length: 2301, dtype: int64

In [7]:
bo_data['income'].astype('int64').abs() # abs - метод возведения в модуль

0       162510
1       285140
2       186030
3       123540
4       200020
         ...  
2296    162510
2297    162510
2298    162510
2299    162510
2300    150120
Name: income, Length: 2301, dtype: int64

**Поиск дубликатов с учетом регистра**

Проверим уникальные категории в столбце `age_group`.

In [8]:
bo_data['age_group'].unique()

array(['Пожилой', 'пожилой', 'Зрелый', 'Молодой', 'ЗРЕЛЫЙ', 'ПОЖИЛОЙ',
       'зрелый', 'молодой', 'МОЛОДОЙ'], dtype=object)

Приведем их к нижнему регистру.

In [9]:
bo_data['age_group'] = bo_data['age_group'].str.lower()

**Обработка дубликатов**

Установим количество явных дубликатов. Если найдутся, то удалим и снова проверим, что их не осталось.

In [10]:
bo_data.duplicated().sum()

7

In [11]:
bo_data = bo_data.drop_duplicates().reset_index(drop=True)

In [12]:
bo_data.duplicated().sum()

0

**Стэмминг**

Посчитаем уникальные значения в столбце `cause`.

In [13]:
bo_data['cause'].value_counts()

кража автомобиля               112
травма во время отдыха         107
сломала ногу в отпуске          95
травма в отпуске                94
украли багаж в отпуске          94
упало дерево на автомобиль      94
сгорел дом                      93
несчастный случай в отпуске     91
сгорела квартира                91
замкнуло электричество дома     91
врезались в автомобиль          91
сотрясение во время отдыха      90
проникли в жилье                89
повреждение автомобиля          89
потоп в квартире                88
хищение автомобиля (каско)      88
повреждение авто                85
несчастный случай на отдыхе     85
кража авто                      82
обокрали дачу                   81
поломка авто                    81
кража в квартире                79
украли авто                     78
взломали квартиру               77
дтп                             76
затопило дом                    73
Name: cause, dtype: int64

Выделим очевидные разные стеммы:  
* дом/квартира/жилье/дача
* авто/дтп
* отдых/отпуск

Из них можно выделить следующие виды страхования:
* страхование недвижимости
* страхование авто
* несчастные случае в отпуске
                                                  

Напишем функцию, которая принимает неизменяемую часть слова и возвращает все названия категорий с ним. Сначала попробуем стемминг и если не поймаем «чужих» названий — значит этот метод подойдет.

In [14]:
def stemming(stem):
    stemmer = SnowballStemmer(language='russian')
    list_of_cat = []
    cat_list = bo_data['cause'].drop_duplicates() # получаем уникальные названия случаев
    for cat in cat_list:
        for word in cat.split(): # находим слово дробя строку на элементы в список
            if stem == stemmer.stem(word):
                list_of_cat.append(cat)
    return list_of_cat
stemming('автомобил') ## проверил, работает во всех случаев, кроме того, если ввожу в качестве стема 'авто', нужна пояснительная бригада

['повреждение автомобиля',
 'упало дерево на автомобиль',
 'хищение автомобиля (каско)',
 'врезались в автомобиль',
 'кража автомобиля']

**Категоризация данных**

Cловарь — распространенный способ хранения информации, который помогает убрать текстовые параметры из основной таблицы и увеличить эффективность работы с данными. Выделим словарь для возрастных групп.

In [15]:
pd.pivot_table(bo_data, index='age_group', values='age_id')

,age_id
age_group,
зрелый,3
молодой,2
пожилой,4


Категоризируем причины выплаты страховых платежей с помощью лемматизации.

In [16]:
m = Mystem()

def lem(row):
    lemmas = m.lemmatize(row)
    print(lemmas)
    if 'авто' in lemmas or 'дтп' in lemmas or 'автомобиль' in lemmas: return 'Случаи с автомобилями'
    elif 'отдых' in lemmas or 'отпуск' in lemmas: return 'Случаи во время отпуска'
    elif 'дом' in lemmas or 'дача' in lemmas or 'квартира' in lemmas or 'жилье' in lemmas: return 'Случаи с недвижимостью'
        
bo_data['cat_cause'] = bo_data['cause'].apply(lem)

Installing mystem to /Users/dsd/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


['повреждение', ' ', 'автомобиль', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['сгорать', ' ', 'квартира', '\n']
['сгорать', ' ', 'дом', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['кража', ' ', 'автомобиль', '\n']
['украсть', ' ', 'авто', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['повреждение', ' ', 'авто', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['украсть', ' ', 'авто', '\n']
['сгорать', ' ', 'дом', '\n']
[

['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['украсть', ' ', 'авто', '\n']
['кража', ' ', 'автомобиль', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['кража', ' ', 'в', ' ', 'квартира', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['взламывать', ' ', 'квартира', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['несчастный', ' ', 'случай', ' ', 'на', ' ', 'отдых', '\n']
['затапливать', ' ', 'дом', '\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['сгорать', ' ', 'квартира', '\n']
['сгорать', ' ', 'квартира', '\n']
['дтп', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['сгорать', ' ', 'квартира', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['повре

['повреждение', ' ', 'автомобиль', '\n']
['сгорать', ' ', 'дом', '\n']
['сгорать', ' ', 'дом', '\n']
['сгорать', ' ', 'дом', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['сгорать', ' ', 'квартира', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['дтп', '\n']
['затапливать', ' ', 'дом', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['кража', ' ', 'автомобиль', '\n']
['сгорать', ' ', 'квартира', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['сгорать', ' ', 'дом', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['взламывать', ' ', 'к

['несчастный', ' ', 'случай', ' ', 'на', ' ', 'отдых', '\n']
['украсть', ' ', 'авто', '\n']
['дтп', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['обкрадывать', ' ', 'дача', '\n']
['повреждение', ' ', 'авто', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['сгорать', ' ', 'дом', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['кража', ' ', 'авто', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['украсть', ' ', 'авто', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['поломка', ' ', 'авто', '\n']
['кража', ' ', 'автомобиль', '\n']
['дтп', '\n']
['несчастный', ' ', 'случай', ' ', 'на', ' ', 'отдых', '\n']
['несчастный', ' ', 'случай', ' ', 'на', ' ', 'отдых', '\n']
['сгорать', ' ', 'дом', '\n']
['

['потоп', ' ', 'в', ' ', 'квартира', '\n']
['затапливать', ' ', 'дом', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['сгорать', ' ', 'квартира', '\n']
['сгорать', ' ', 'квартира', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['затапливать', ' ', 'дом', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['несчастный', ' ', 'случай', ' ', 'на', ' ', 'отдых', '\n']
['повреждение', ' ', 'авто', '\n']
['несчастный', ' ', 'случай', ' ', 'на', ' ', 'отдых', '\n']
['кража', ' ', 'в', ' ', 'квартира', '\n']
['несчастный', ' ', 'случай', ' ', 'на', ' ', 'отдых', '\n']
['повреждение', ' ', 'авто', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['кража', ' ', 'автомобиль', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['упасть', ' ', 'дерево', ' ', 'н

['кража', ' ', 'автомобиль', '\n']
['взламывать', ' ', 'квартира', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['сгорать', ' ', 'квартира', '\n']
['украсть', ' ', 'авто', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['кража', ' ', 'автомобиль', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['сгорать', ' ', 'дом', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['кража', ' ', 'в', ' ', 'квартира', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['сгорать', ' ', 'дом', '\n']
['сгорать', ' ', 'квартира', '\n']
['кража', ' ', 'автомобиль', '\n']
['сгорать', ' ', 'квартира', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['взламывать', ' '

['проникать', ' ', 'в', ' ', 'жилье', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['обкрадывать', ' ', 'дача', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['взламывать', ' ', 'квартира', '\n']
['поломка', ' ', 'авто', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['кража', ' ', 'авто', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['поломка', ' ', 'авто', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['сгорать', ' ', 'дом', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['кража', ' ', 'автомобиль', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['сгорать', ' ', 'дом', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['хищение', ' ', 'автомобиль', ' (', 

['сгорать', ' ', 'квартира', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['кража', ' ', 'авто', '\n']
['замыкать', ' ', 'электричество', ' ', 'дом', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['сгорать', ' ', 'дом', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['кража', ' ', 'авто', '\n']
['повреждение', ' ', 'авто', '\n']
['сгорать', ' ', 'дом', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['сотрясение', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['кража', ' ', 'автомобиль', '\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['травма', ' ', 'в', ' ', 'отпуск', '\n']
['сгорать', ' ', 'дом', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['повреждение', ' ', 'авто', '\n']
['несчастный', ' ', 'случай', ' ', 'в', ' '

Категоризируем статус выплаты компенсации — 0 выплат, выплачена часть, полностью выплачена.

In [27]:
def cat_compensation(paid):
    if paid == 1:
        return 'Полностью выплачено'
    elif paid == 0:
        return '0 выплат'
    else: 
        return 'Выплачена часть'
bo_data['cat_compensation'] = bo_data['perc_of_compensation_paid'].apply(cat_compensation)

Категоризируем `income`, разбив данные по квантилям.

In [32]:
bo_data['quant'] = pd.qcut(bo_data['income'], 5)


## Результаты

In [33]:
bo_data

,id,perc_of_compensation_paid,age_in_days,age_id,age_group,income,count_3-6_months_late,count_6-12_months_late,count_more_than_12_months_late,cause,cat_cause,cat_compensation,quant
0,649,0.001,27384.0,4,пожилой,162510.0,0,0,0,повреждение автомобиля,Случаи с автомобилями,Выплачена часть,"(141052.0, 187640.0]"
1,81136,0.124,23735.0,4,пожилой,285140.0,0,0,0,упало дерево на автомобиль,Случаи с автомобилями,Выплачена часть,"(267628.0, 2700040.0]"
2,70762,1.000,17170.0,3,зрелый,-186030.0,0,0,0,сломала ногу в отпуске,Случаи во время отпуска,Полностью выплачено,"(-437570.001, 91458.0]"
3,53935,0.198,16068.0,3,зрелый,-123540.0,0,0,0,хищение автомобиля (каско),Случаи с автомобилями,Выплачена часть,"(-437570.001, 91458.0]"
4,15476,0.041,10591.0,2,молодой,-200020.0,1,0,0,несчастный случай в отпуске,Случаи во время отпуска,Выплачена часть,"(-437570.001, 91458.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2289,16440,0.474,18628.0,3,зрелый,162510.0,0,0,0,дтп,Случаи с автомобилями,Выплачена часть,"(141052.0, 187640.0]"
2290,109416,0.413,17525.0,3,зрелый,162510.0,0,0,0,кража в квартире,Случаи с недвижимостью,Выплачена часть,"(141052.0, 187640.0]"
2291,4268,0.109,21906.0,4,пожилой,162510.0,0,0,0,проникли в жилье,Случаи с недвижимостью,Выплачена часть,"(141052.0, 187640.0]"
2292,85708,0.036,25925.0,4,пожилой,162510.0,0,0,0,обокрали дачу,Случаи с недвижимостью,Выплачена часть,"(141052.0, 187640.0]"


Подготовим 3 таблицы и изучим результат.

*ответьте на вопрос — есть ли зависимость между возрастной группой и статусом выплаты?*

зависимости между возрастной группы и статусом выплаты нет, хотя можем наблюдать, что группа ЗРЕЛЫЙ имеет больше выплат, нежели группа МОЛОДОЙ, но так же у них отличается количество выплат в целом, такая разность выходит как раз из-за этого

In [54]:
pd.pivot_table(bo_data, index='age_group', values='id', columns='cat_compensation', aggfunc='count')

cat_compensation,0 выплат,Выплачена часть,Полностью выплачено
age_group,,,
зрелый,62,884,100
молодой,31,206,52
пожилой,57,698,204


In [55]:
count_of_zrelii = bo_data[bo_data['age_group'] == 'зрелый'].count()[0]
count_of_molodoy = bo_data[bo_data['age_group'] == 'молодой'].count()[0]


print(f'Разность между группой зрелых людей и молодых: {count_of_zrelii - count_of_molodoy}')

Разность между группой зрелых людей и молодых: 757


*есть ли зависимость между причиной страхового случая и статусом выплаты?*

можем наблюдать отсутсвие зависимости между причиной страхового случая и статуса выплаты


In [45]:
pd.pivot_table(bo_data, index='cat_cause', values='id', columns='cat_compensation', aggfunc='count')

cat_compensation,0 выплат,Выплачена часть,Полностью выплачено
cat_cause,,,
Случаи во время отпуска,42,536,78
Случаи с автомобилями,54,640,182
Случаи с недвижимостью,54,612,96


*есть ли зависимость между доходом клиента и статусом выплаты?*

по данной таблице можем наблюдать отсутствие зависимости между доходом и статусом выплаты


In [47]:
pd.pivot_table(bo_data, index='quant', values='id', columns='cat_compensation', aggfunc='count')

cat_compensation,0 выплат,Выплачена часть,Полностью выплачено
quant,,,
"(-437570.001, 91458.0]",44,319,96
"(91458.0, 141052.0]",31,361,67
"(141052.0, 187640.0]",31,369,59
"(187640.0, 267628.0]",20,371,67
"(267628.0, 2700040.0]",24,368,67
